## Homework

In this homework, we'll deploy the Straight vs Curly Hair Type model we trained in the 
[previous homework](../08-deep-learning/homework.md).

Download the model from here: 

https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras



## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

* 27 Mb
* 43 Mb
* **77 Mb**
* 127 Mb

In [14]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.lite as tflite

import numpy as np

In [2]:
model = keras.models.load_model('model_2024_hairstyle.keras')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model_2024_hairstyle.tflite', 'wb') as f_out:
    f_out.write(tflite_model)


INFO:tensorflow:Assets written to: /var/folders/m5/90bb4mps3dv507n82593klgr0000gn/T/tmpezpi6e2b/assets


INFO:tensorflow:Assets written to: /var/folders/m5/90bb4mps3dv507n82593klgr0000gn/T/tmpezpi6e2b/assets


Saved artifact at '/var/folders/m5/90bb4mps3dv507n82593klgr0000gn/T/tmpezpi6e2b'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  13458110992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13458111952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13458110032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13458112336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13459720144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13459720720: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1735862784.480385 33718271 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1735862784.481020 33718271 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2025-01-02 16:06:24.482524: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/m5/90bb4mps3dv507n82593klgr0000gn/T/tmpezpi6e2b
2025-01-02 16:06:24.483264: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-01-02 16:06:24.483271: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/m5/90bb4mps3dv507n82593klgr0000gn/T/tmpezpi6e2b
2025-01-02 16:06:24.486730: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2025-01-02 16:06:24.487348: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-01-02 16:06:24.591231: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle a

In [6]:
!ls -lh

total 484656
-rw-r--r--  1 koda  staff   5.0K Jan  2 16:06 hw9.ipynb
-rw-r--r--@ 1 koda  staff   153M Jan  2 16:01 model_2024_hairstyle.keras
-rw-r--r--@ 1 koda  staff    77M Jan  2 16:06 model_2024_hairstyle.tflite



## Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output. 

What's the output index for this model?

* 3
* 7
* **13**
* 24

In [8]:
interpreter = tflite.Interpreter(model_path='model_2024_hairstyle.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
output_index

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


13


## Preparing the image

You'll need some code for downloading and resizing images. You can use 
this code:

```python
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
```

For that, you'll need to have `pillow` installed:

```bash
pip install pillow
```

Let's download and resize this image: 

https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

Based on the previous homework, what should be the target size for the image?

In [11]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [13]:
image = download_image('https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg')
image = prepare_image(image, (200, 200))


## Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* **0.24**
* 0.44
* 0.64
* 0.84

In [15]:
def preprocess_image(x):
    return x / 255

In [16]:
x = np.array(image, dtype='float32')
X = np.array([x])

X = preprocess_image(X)


In [19]:
X[0][0][0]

array([0.23921569, 0.40784314, 0.08627451], dtype=float32)



## Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.293
* 0.493
* 0.693
* **0.893**

In [20]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
preds

array([[0.8937725]], dtype=float32)


## Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will 
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook 
by importing the file and invoking the function from this file.  


## Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.10

COPY model_2024_hairstyle_v2.tflite .

RUN pip install numpy==1.23.1
```

Note that it uses Python 3.10. The latest models of TF Lite
do not support Python 3.12 yet, so we need to use 3.10. Also,
for this part, we will use TensorFlow 2.14.0. We have tested
it, and the models created with 2.17 could be served with 2.14.0.

For that image, we also needed to use an older version of numpy
(1.23.1)

The docker image is published to [`agrigorev/model-2024-hairstyle:v3`](https://hub.docker.com/r/agrigorev/model-2024-hairstyle/tags).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.
* The wheel for this combination that you'll need to use in your Docker image is https://github.com/alexeygrigorev/tflite-aws-lambda/raw/main/tflite/tflite_runtime-2.14.0-cp310-cp310-linux_x86_64.whl



## Question 5

Download the base image `agrigorev/model-2024-hairstyle:v3`. You can do it with [`docker pull`](https://docs.docker.com/engine/reference/commandline/pull/).

So what's the size of this base image?

* 182 Mb
* 382 Mb
* 582 Mb
* **782 Mb**

You can get this information when running `docker images` - it'll be in the "SIZE" column.

## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `model_2024_hairstyle_v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference). 
The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

What's the output from the model?

* 0.229
* **0.429**
* 0.629
* 0.829

In [30]:
import requests

url = 'http://localhost:8080/2015-03-31/functions/function/invocations'

data = {'url': 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'}

result = requests.post(url, json=data).json()
print(result)

0.42985355854034424
